In [1]:
#All necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
#A progress bar indicator
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
import keras
import cv2
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

c:\users\ammuuu\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\ammuuu\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\ammuuu\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
# set a random seed, so that code can be run against same input
np.random.seed(42)
tf.random.set_seed(42)
#fixed constants
IMAGE_SIZE = 224
BATCH_SIZE = 32
TEST_SIZE = 0.25

In [5]:
train_df = pd.read_csv("dataset/train.csv")
train_df.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [6]:
train_df['diagnosis'].value_counts()

0    1805
2     999
1     370
4     295
3     193
Name: diagnosis, dtype: int64

In [14]:
def get_pad_width(image, new_shape, is_rgb=True):
    #Pad image to get same sizing
    pad_diff = new_shape - image.shape[0], new_shape - image.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def crop_image_from_gray(image, tol=7):
    #Crop images to obtain section of only eye
    if image.ndim == 2:
        mask = image > tol
        return image[np.ix_(mask.any(1), mask.any(0))]
    elif image.ndim == 3:
        gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
      
        check_shape = image[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        #If the image was too dark, don't do anything to the original one
        if (check_shape == 0):
            return image 
        else:
            img1=image[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2=image[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3=image[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            image = np.stack([img1, img2, img3], axis=-1)
        return image

def load_ben_color(image, sigmaX):
    #convert image color to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), sigmaX),-4, 128)
    return image

def preprocess_image(image_path, desired_size = 224):
    #Resize the image and add perform all required preprocessing
    image = cv2.imread(image_path)
    image = load_ben_color(image, sigmaX = 30)
    image = cv2.resize(image, (desired_size, desired_size))
    image[:, :, 0] = image[:, :, 1]
    image[:, :, 2] = image[:, :, 1]
    return image

In [24]:
#Do all necessary preprocessing and place in array
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image(f'dataset/images/{image_id}.png')

100%|██████████| 3662/3662 [57:19<00:00,  1.06it/s]  


In [25]:
#one-hot encode
y_train = pd.get_dummies(train_df['diagnosis']).values

In [26]:
#The following generates multi label classification
#This is done because if for instance an image is of level 4
#It is related to all the other levels before it
#IOW the levels aren't mutually exclusive
#Ex: - generally lv 4 -> [0, 0, 0, 1]; in this case [1, 1, 1, 1]
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel y_train:", y_train_multi.sum(axis=0))

Original y_train: [1805  370  999  193  295]
Multilabel y_train: [3662 1857 1487  488  295]


In [28]:
#split dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_multi, test_size=TEST_SIZE, random_state=42)

In [42]:
#Image data generator for data Augmentation
def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  
        fill_mode='constant',
        cval=0.,  
        horizontal_flip=True,
        vertical_flip=True,
    )

# Using original generator
data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE, seed=42)

In [43]:
#Create the DensetNet121 model, loading a pretrained weight to prevent unnecessary training
base_model = DenseNet121(include_top=False, weights="../pretrained_weights/densenet_weights/notop.h5", input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))

dense = keras.models.Sequential()
dense.add(base_model)
dense.add(keras.layers.GlobalAveragePooling2D())
dense.add(keras.layers.Dropout(0.5))
dense.add(keras.layers.Dense(5, activation='sigmoid'))

dense.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.00005), metrics=['accuracy'])

In [44]:
#Callback functions, to prevent overfitting, and model checkpointing
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto', verbose=1)
model_checkpoint = ModelCheckpoint('models/dense_model_{epoch:02d}-{val_accuracy:.2f}ML.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only = True)
lst_callbacks = [early_stopper, learning_rate_reduction, model_checkpoint]

In [45]:
history = dense.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=15,
    validation_data=(x_val, y_val),
    callbacks=lst_callbacks
)

Epoch 1/15
86/85 [==============================] - ETA: -4s - loss: 0.4251 - accuracy: 0.4836
Epoch 00001: val_accuracy improved from inf to 0.70197, saving model to dense_model_01-0.70.h5
86/85 [==============================] - 2426s 28s/step - loss: 0.4251 - accuracy: 0.4836 - val_loss: 0.4322 - val_accuracy: 0.7020
Epoch 2/15
86/85 [==============================] - ETA: -4s - loss: 0.2208 - accuracy: 0.6256
Epoch 00002: val_accuracy improved from 0.70197 to 0.60699, saving model to dense_model_02-0.61.h5
86/85 [==============================] - 2110s 25s/step - loss: 0.2208 - accuracy: 0.6256 - val_loss: 0.2511 - val_accuracy: 0.6070
Epoch 3/15
86/85 [==============================] - ETA: -3s - loss: 0.1709 - accuracy: 0.6271  ETA: 6
Epoch 00003: val_accuracy improved from 0.60699 to 0.57314, saving model to dense_model_03-0.57.h5
86/85 [==============================] - 1881s 22s/step - loss: 0.1709 - accuracy: 0.6271 - val_loss: 0.1890 - val_accuracy: 0.5731
Epoch 4/15
86/85 [

In [46]:
dense.save("models/densenet121ML.h5")

In [47]:
y_val_pred = dense.predict(x_val)

In [48]:
#the y value is considered if its greater than a specific threshold, in this case 0.5
val_y = y_val_pred > 0.5
#since we created multi labels, we can sum up the individual 1s to obtain the predicted class
val_y = val_y.astype(int).sum(axis=1) - 1

Val_y has the predicted classes

<h1>Evaluation Section</h1>

In [50]:
#we split this into variables, such that d will obtain the real y values in the validation set
#a, b => Train
#c, d => Validation (c -> id_code, d -> diagnosis)
a, b, c, d = train_test_split(train_df['id_code'], train_df['diagnosis'], test_size=TEST_SIZE, random_state=42)

In [51]:
real_y = list(d)

In [52]:
actual = real_y
predicted = val_y
results = confusion_matrix(actual, predicted) 
  
print ('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ')
print (classification_report(actual, predicted))

Confusion Matrix :
[[443   6   1   0   0]
 [ 13  51  17   0   0]
 [  6  40 170  34   7]
 [  0   1  13  15  15]
 [  0   3  26  19  36]]
Accuracy Score : 0.7805676855895196
Report : 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       450
           1       0.50      0.63      0.56        81
           2       0.75      0.66      0.70       257
           3       0.22      0.34      0.27        44
           4       0.62      0.43      0.51        84

    accuracy                           0.78       916
   macro avg       0.61      0.61      0.60       916
weighted avg       0.79      0.78      0.78       916

